## Imports

In [26]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [27]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 30th Jan. 6th


In [50]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [51]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [52]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [53]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [54]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,NaN,Condo/Co-op,398 NE 5th St #4408,Miami,FL,33132.0,775000.0,1.0,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Miami/398-NE-5th-St-...,NaN,NaN,N,Y,25.77888,-80.189056


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [55]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [56]:
df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [57]:
df2.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
2,PAST SALE,2025-01-06,Condo/Co-op,92 SW 3rd St #4411,Miami,FL,33130.0,595000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/92-SW-3rd-St-3...,MARMLS,A11712487,N,Y,25.769593,-80.195047
118,PAST SALE,2025-01-06,Condo/Co-op,151 Crandon Blvd #1031,Key Biscayne,FL,33149.0,1175000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/151-Cra...,MARMLS,A11630561,N,Y,25.696473,-80.161501
213,PAST SALE,2025-01-06,Condo/Co-op,4874 NW 97th Ct #360,Doral,FL,33178.0,300000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/4874-NW-97th-C...,MARMLS,A11573075,N,Y,25.817851,-80.354440
97,PAST SALE,2025-01-06,Condo/Co-op,413 Swallow Dr #9,Miami Springs,FL,33166.0,269000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Springs/413-Sw...,MARMLS,A11654559,N,Y,25.825777,-80.285724
74,PAST SALE,2025-01-06,Condo/Co-op,700 16th St #105,Miami Beach,FL,33139.0,240000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/700-16th...,MARMLS,A11678600,N,Y,25.788886,-80.135575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,PAST SALE,2024-12-30,Condo/Co-op,12850 SW 43rd Dr #155,Miami,FL,33175.0,258500.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/12850-SW-43rd-...,MARMLS,A11706475,N,Y,25.727143,-80.403041
34,PAST SALE,2024-12-30,Condo/Co-op,2415 NW 16th St Rd #416,Miami,FL,33125.0,275000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2415-NW-16th-S...,MARMLS,A11687193,N,Y,25.789892,-80.236447
35,PAST SALE,2024-12-30,Condo/Co-op,3800 Palm Ave #209,Hialeah,FL,33012.0,228000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/3800-Palm-Av...,MARMLS,A11668122,N,Y,25.857271,-80.283055
58,PAST SALE,2024-12-30,Condo/Co-op,160 NE 203rd Ter #30,Miami Gardens,FL,33179.0,85000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/160-NE-203rd-T...,MARMLS,A11692517,N,Y,25.961793,-80.199898


In [58]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [59]:
df_top_ten = df2.head(10)

## Collect Agent Information

In [60]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [61]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [62]:
response_df = pd.DataFrame(response_list)

In [63]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [64]:
df_top_ten = merged_df

## Current Week's Values

In [131]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

92


In [132]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$646,506


In [133]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$511


In [134]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$59,478,570


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [74]:
# Define the date range
start_date = '2024-12-23'
end_date = '2024-12-29'

# Filter the DataFrame between two dates (inclusive)
old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [75]:
# comparison week total sales
print(f'Comparison week filtered sales: {(len(old_week))}')
print('-----------')
print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
print('-----------')
print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
print('-----------')
print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
print('-----------')

Comparison week filtered sales: 103
-----------
Comparison week average sales price: $626,798.25
-----------
Comparison week average psf: $472.69
-----------
Comparison week sales total: $64,560,220.00
-----------


In [76]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
103
Input Previous Week Condo Average Sales Price:
626798
Input Previous Week Condo Average PSF:
473
Input Previous Week Condo Sales total (ex: 198_000_000)
64560220


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [77]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [78]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [79]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [80]:
pd.set_option('display.max_columns',None)

In [81]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange


In [82]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [83]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange
1,PAST SALE,2024-12-30,Condo/Co-op,16051 Collins Ave #703,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,4.5,TURNBERRY OCEAN COLONY,3480.0,NaN,2007.0,NaN,1351.0,6008.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11645549,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Lana Bell,Douglas Elliman,None,None,Lana Bell,Douglas Elliman,None,None,2,blue
2,PAST SALE,2024-12-31,Condo/Co-op,4701 N Meridian Ave #125,Miami Beach,FL,33140.0,3750000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2401.0,NaN,2019.0,NaN,1562.0,10678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11476701,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Pedro Sousa Azevedo,HAUS Realty,None,None,Pablo Buttice,GZI Real Estate Sales & Market,None,None,3,blue
3,PAST SALE,2024-12-30,Condo/Co-op,2678 Tigertail Ave Unit 202T,Miami,FL,33133.0,2050000.0,2.0,2.5,Mr. C Residences,1649.0,NaN,2024.0,NaN,1243.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11684294,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Lani Kahn Drody,Lowell International Rlty LLC,Lauren Uzarek,Lowell International Rlty LLC,Amner Pereira,Engel & Völkers Miami,None,None,4,blue
4,PAST SALE,2025-01-03,Condo/Co-op,13647 Deering Bay Dr Ph 151,Coral Gables,FL,33158.0,1860000.0,3.0,2.5,DEERING BAY CONDO I,2750.0,NaN,1993.0,NaN,676.0,2476.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13647-D...,MARMLS,A11638195,N,Y,25.634738,-80.291339,https://www.redfin.com/FL/Coral-Gables/13647-D...,Adam Levy,"Compass Florida, LLC.",Chris Crystal,Miami Real Estate Company,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue
5,PAST SALE,2024-12-30,Condo/Co-op,6365 Collins Ave #4007,Miami Beach,FL,33141.0,1775000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1343.0,1635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11671753,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Maria Carrillo,The Keyes Company,None,None,Maria Carrillo,The Keyes Company,None,None,6,blue
6,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,7,blue
7,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None,8,blue
8,PAST SALE,2025-01-06,Condo/Co-op,151 Crandon Blvd #1031,Key Biscayne,FL,33149.0,1175000.0,2.0,2.0,THE EMERALDBAY @ KEY COLO,1547.0,NaN,1980.0,NaN,760

In [84]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [85]:
m.save('index.html')

## Data snagger

In [86]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [87]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [88]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_01062024


# CREATE TEMPLATE 

In [89]:
muni_set = set(df_top_ten['CITY'])

In [90]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [91]:
df_top_ten.reset_index(inplace=True,drop=True)

In [92]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [93]:
top_sale

'OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit 1206N in Key Biscayne'

In [94]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [95]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [96]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
1,PAST SALE,2024-12-30,Condo/Co-op,16051 Collins Ave #703,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,4.5,TURNBERRY OCEAN COLONY,3480.0,NaN,2007.0,NaN,1351.0,6008.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11645549,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Lana Bell,Douglas Elliman,None,None,Lana Bell,Douglas Elliman,None,None,2,blue,TURNBERRY OCEAN COLONY at 16051 Collins Ave #7...
2,PAST SALE,2024-12-31,Condo/Co-op,4701 N Meridian Ave #125,Miami Beach,FL,33140.0,3750000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2401.0,NaN,2019.0,NaN,1562.0,10678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11476701,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Pedro Sousa Azevedo,HAUS Realty,None,None,Pablo Buttice,GZI Real Estate Sales & Market,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2024-12-30,Condo/Co-op,2678 Tigertail Ave Unit 202T,Miami,FL,33133.0,2050000.0,2.0,2.5,Mr. C Residences,1649.0,NaN,2024.0,NaN,1243.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11684294,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Lani Kahn Drody,Lowell International Rlty LLC,Lauren Uzarek,Lowell International Rlty LLC,Amner Pereira,Engel & Völkers Miami,None,None,4,blue,Mr. C Residences at 2678 Tigertail Ave Unit 20...
4,PAST SALE,2025-01-03,Condo/Co-op,13647 Deering Bay Dr Ph 151,Coral Gables,FL,33158.0,1860000.0,3.0,2.5,DEERING BAY CONDO I,2750.0,NaN,1993.0,NaN,676.0,2476.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13647-D...,MARMLS,A11638195,N,Y,25.634738,-80.291339,https://www.redfin.com/FL/Coral-Gables/13647-D...,Adam Levy,"Compass Florida, LLC.",Chris Crystal,Miami Real Estate Company,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,DEERING BAY CONDO I at 13647 Deering Bay Dr Ph...
5,PAST SALE,2024-12-30,Condo/Co-op,6365 Collins Ave #4007,Miami Beach,FL,33141.0,1775000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1343.0,1635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11671753,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Maria Carrillo,The Keyes Company,None,None,Maria Carrillo,The Keyes Company,None,None,6,blue,AKOYA CONDO at 6365 Collins Ave #4007 in Miami...
6,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gab

In [135]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,150,000 to $5,850,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Key Biscayne, Coral Gables, Miami Beach, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 92 condo sales totaling $59,478,570 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 103 condo sales totaling $64,560,220.

Last week’s units sold for an average of $646,506, higher than the $626,798 average price fr

In [98]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [99]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [100]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [101]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
1,PAST SALE,2024-12-30,Condo/Co-op,16051 Collins Ave #703,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,4.5,TURNBERRY OCEAN COLONY,3480.0,NaN,2007.0,NaN,1351.0,6008.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11645549,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Lana Bell,Douglas Elliman,None,None,Lana Bell,Douglas Elliman,None,None,2,blue,TURNBERRY OCEAN COLONY at 16051 Collins Ave #7...
2,PAST SALE,2024-12-31,Condo/Co-op,4701 N Meridian Ave #125,Miami Beach,FL,33140.0,3750000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2401.0,NaN,2019.0,NaN,1562.0,10678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11476701,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Pedro Sousa Azevedo,HAUS Realty,None,None,Pablo Buttice,GZI Real Estate Sales & Market,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2024-12-30,Condo/Co-op,2678 Tigertail Ave Unit 202T,Miami,FL,33133.0,2050000.0,2.0,2.5,Mr. C Residences,1649.0,NaN,2024.0,NaN,1243.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11684294,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Lani Kahn Drody,Lowell International Rlty LLC,Lauren Uzarek,Lowell International Rlty LLC,Amner Pereira,Engel & Völkers Miami,None,None,4,blue,Mr. C Residences at 2678 Tigertail Ave Unit 20...
4,PAST SALE,2025-01-03,Condo/Co-op,13647 Deering Bay Dr Ph 151,Coral Gables,FL,33158.0,1860000.0,3.0,2.5,DEERING BAY CONDO I,2750.0,NaN,1993.0,NaN,676.0,2476.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13647-D...,MARMLS,A11638195,N,Y,25.634738,-80.291339,https://www.redfin.com/FL/Coral-Gables/13647-D...,Adam Levy,"Compass Florida, LLC.",Chris Crystal,Miami Real Estate Company,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,DEERING BAY CONDO I at 13647 Deering Bay Dr Ph...
5,PAST SALE,2024-12-30,Condo/Co-op,6365 Collins Ave #4007,Miami Beach,FL,33141.0,1775000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1343.0,1635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11671753,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Maria Carrillo,The Keyes Company,None,None,Maria Carrillo,The Keyes Company,None,None,6,blue,AKOYA CONDO at 6365 Collins Ave #4007 in Miami...
6,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gab

In [102]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,150,000 to $5,850,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Key Biscayne, Coral Gables, Miami Beach, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 702 condo sales totaling $537,381,450 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 103 condo sales totaling $64,560,220.

Last week’s units sold for an average of $765,501, higher than the $626,798 average price 

In [110]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Key-Biscayne/350-Ocean-Dr-33149/unit-1206N/home/79138168


In [111]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16051-Collins-Ave-33160/unit-703/home/43470046


In [112]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/200-Biscayne-Boulevard-Way-33131/unit-4408/home/42678700


In [113]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Key-Biscayne/151-Crandon-Blvd-33149/unit-1031/home/42924909


In [114]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Key-Biscayne/350-Ocean-Dr-33149/unit-1206N/home/79138168


In [115]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/19955-NE-38th-Ct-33180/unit-2401/home/43257107


In [116]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
1,PAST SALE,2024-12-30,Condo/Co-op,16051 Collins Ave #703,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,4.5,TURNBERRY OCEAN COLONY,3480.0,NaN,2007.0,NaN,1351.0,6008.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11645549,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Lana Bell,Douglas Elliman,None,None,Lana Bell,Douglas Elliman,None,None,2,blue,TURNBERRY OCEAN COLONY at 16051 Collins Ave #7...
2,PAST SALE,2024-12-31,Condo/Co-op,4701 N Meridian Ave #125,Miami Beach,FL,33140.0,3750000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2401.0,NaN,2019.0,NaN,1562.0,10678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11476701,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Pedro Sousa Azevedo,HAUS Realty,None,None,Pablo Buttice,GZI Real Estate Sales & Market,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2024-12-30,Condo/Co-op,2678 Tigertail Ave Unit 202T,Miami,FL,33133.0,2050000.0,2.0,2.5,Mr. C Residences,1649.0,NaN,2024.0,NaN,1243.0,3524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2678-Tigertail...,MARMLS,A11684294,N,Y,25.731339,-80.236374,https://www.redfin.com/FL/Miami/2678-Tigertail...,Lani Kahn Drody,Lowell International Rlty LLC,Lauren Uzarek,Lowell International Rlty LLC,Amner Pereira,Engel & Völkers Miami,None,None,4,blue,Mr. C Residences at 2678 Tigertail Ave Unit 20...
4,PAST SALE,2025-01-03,Condo/Co-op,13647 Deering Bay Dr Ph 151,Coral Gables,FL,33158.0,1860000.0,3.0,2.5,DEERING BAY CONDO I,2750.0,NaN,1993.0,NaN,676.0,2476.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/13647-D...,MARMLS,A11638195,N,Y,25.634738,-80.291339,https://www.redfin.com/FL/Coral-Gables/13647-D...,Adam Levy,"Compass Florida, LLC.",Chris Crystal,Miami Real Estate Company,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,DEERING BAY CONDO I at 13647 Deering Bay Dr Ph...
5,PAST SALE,2024-12-30,Condo/Co-op,6365 Collins Ave #4007,Miami Beach,FL,33141.0,1775000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1343.0,1635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11671753,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Maria Carrillo,The Keyes Company,None,None,Maria Carrillo,The Keyes Company,None,None,6,blue,AKOYA CONDO at 6365 Collins Ave #4007 in Miami...
6,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gab

## Time on Market Calculator

In [129]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 1) ## List (Earlier) date
date2 = datetime(2024, 12, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

151


## Clean CSV for Datawrapper Chart

In [117]:
chart_df = df_top_ten

In [118]:
chart_df = chart_df.fillna(" ")

In [119]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [120]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [121]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [122]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [123]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [124]:
csv_date_string = today.strftime("%m_%d_%Y")

In [125]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [126]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-06,350 Ocean Dr Unit 1206N in Key Biscayne,"$5,850,000",3,4.5,"2,257",2014,"$2,592",MARMLS,A11602978,Giulietta Ulloa BHHS EWM Realty Jennifer Perez...,Elena Chacon BHHS EWM Realty
1,2024-12-30,16051 Collins Ave #703 in Sunny Isles Beach,"$4,700,000",3,4.5,"3,480",2007,"$1,351",MARMLS,A11645549,Lana Bell Douglas Elliman,Lana Bell Douglas Elliman
2,2024-12-31,4701 N Meridian Ave #125 in Miami Beach,"$3,750,000",2,2.5,"2,401",2019,"$1,562",MARMLS,A11476701,Pedro Sousa Azevedo HAUS Realty,Pablo Buttice GZI Real Estate Sales & Market
3,2024-12-30,2678 Tigertail Ave Unit 202T in Miami,"$2,050,000",2,2.5,"1,649",2024,"$1,243",MARMLS,A11684294,Lani Kahn Drody Lowell International Rlty LLC ...,Amner Pereira Engel & Völkers Miami
4,2025-01-03,13647 Deering Bay Dr Ph 151 in Coral Gables,"$1,860,000",3,2.5,"2,750",1993,$676,MARMLS,A11638195,"Adam Levy Compass Florida, LLC. Chris Crystal ...",NON-MLS MEMBER MAR NON MLS MEMBER
5,2024-12-30,6365 Collins Ave #4007 in Miami Beach,"$1,775,000",2,2.0,"1,322",2004,"$1,343",MARMLS,A11671753,Maria Carrillo The Keyes Company,Maria Carrillo The Keyes Company
6,2025-01-06,19955 NE 38th Ct #2401 in Aventura,"$1,550,000",2,3.5,"2,400",2000,$646,MARMLS,A11590324,"Karen Matluck Compass Florida, LLC Valentina G...",Lana Bell Douglas Elliman
7,2025-01-06,301 Altara Ave Unit LPH912 in Coral Gables,"$1,470,000",3,3.5,"1,539",2019,$955,MARMLS,A11637836,Casey Loundy One Sotheby's International Re,Ludmila Bogatov Bogatov Realty
8,2025-01-06,151 Crandon Blvd #1031 in Key Biscayne,"$1,175,000",2,2.0,"1,547",1980,$760,MARMLS,A11630561,"James Salas E Realty International, Inc.","James Salas E Realty International, Inc."
9,2024-12-30,200 Biscayne Boulevard Way #4408 in Miami,"$1,150,000",2,2.5,"1,500",2008,$767,MARMLS,A11707952,Richard Garcia Global Luxury Realty LLC,Richard Garcia Global Luxury Realty LLC
